In [1]:
import sys

sys.path.append("C:\\Users\\lauth\\OneDrive\\Desktop\\open_ai_assistant_v2")
from src.components.chains.chains import Assistant

assistant = Assistant(conversation_id="8af20de4-cd11-41da-9a9d-04762dd91322")

In [2]:
def chat(user_message: str) -> dict[str, any]:
    # Generamos el resumen
    assistant.add_user_message(message=user_message)
    summary = assistant.predict_summary()
    print("summary")
    print(summary)

    # Generamos el requerimiento
    user_request = assistant.get_request()
    print("user_request")
    print(user_request["request"])

    # Filtramos el requerimiento y obtenemos una respuesta
    first_response = assistant.improved_resquest_filter(
        user_request=user_request["request"],
        coversation_summary=summary,
        user_message=user_message,
    )
    print("first_response")
    print(first_response)

    # Procesamos la respuesta para entregar una respuesta final
    final_response = assistant.process_response(user_message, first_response["response"])
    print("final_response")
    print(final_response)

    # Actualizando el chat
    assistant.add_ai_message(message=final_response["response"])

    response = {
        "summary": summary,
        "request": user_request.get("request", None),
        "request_type": first_response.get("response_type", None),
        "response": first_response.get("response", None),
        "sql": first_response.get("sql", None),
        "dataframe": first_response.get("dataframe", None),
        "final_response": final_response.get("response", None),
        "language": final_response.get("detected_language", None)
    }

    return response



In [7]:
user_last_message = "ahora dame su presion estatica para el mismo periodo"
res = chat(user_last_message)

summary

        The conversation is about the temperature and static pressure of a computer with the tag FQI-EMED-3138.03-057 for the month of August 2023. The average temperature recorded for this computer in August 2023 was 22.1462°C. The human now wants to know the static pressure for the same period.
user_request
The static pressure for the same period as the average temperature of the specified computer tag is not available. Please provide more information or specify a different time period for the request.


C:\Users\lauth\OneDrive\Desktop\open_ai_assistant_v2\src\tools\vanna\vanna_tool.py:40: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql, self.conn)


first_response
{'response_type': 'complex', 'response': "The query requested all columns from the table `dbo_v2.med_sistema_medicion` where the `SamplingPointTag` is 'Pressão Estática (kPa)'.", 'dataframe': Empty DataFrame
Columns: [Id, IdPlataforma_fk, Nombre, Tag, Estado, FechaInicialMuestreo, IdCliente, IdTipoFluido_fk, SubTipoFluido, IdAplicabilidad_fk, SHOW_AUTOMATIC, VarCromatografia, Prover, IsDisponible, IdMeteringStation_fk, IdClase_fk, SamplingPointTag, LimiteEstadisticoInicial, PROFACCODE, SOURCE, RefPid, NumeroLineaProceso, Localizacion, NominalLineSize, IdSchedule_fk, IdRating_fk, ClasificacionArea, Servicio, LimiteEstadistico_Default, LimiteEstadistico_Manual, IdSamplingPoint_fk, IsVisible, usu_creacion, fec_creacion, usu_modificacion, fec_modificacion, Uso, IdArea_fk, NombreSanqGas, IdDeBaseOperacional_fk, EsProvador, Tramo, IdDelSistemaAsociado_fk]
Index: []

[0 rows x 43 columns], 'sql': "SELECT *\nFROM dbo_v2.med_sistema_medicion\nWHERE SamplingPointTag = 'Pressão Est

In [8]:
print(res["sql"])

SELECT *
FROM dbo_v2.med_sistema_medicion
WHERE SamplingPointTag = 'Pressão Estática (kPa)';


In [5]:
from src.components.memory import MEMORY_TYPES

current_summary= assistant.summary
current_messages = assistant.get_current_messages()

print("------------------ current_conversation ------------------")
print(current_summary)
print("\n ------------------ current_messages ------------------")
for message in current_messages:
    if message["type"] == MEMORY_TYPES["AI"]:
        print("AI Message: ", message["content"])
    else:
        print("Human Message: ", message["content"])

------------------ current_conversation ------------------

        The conversation is about obtaining data for a specific computer with the tag FQI-EMED-3138.03-057. The human asks for the average temperature for the month of August 2023, and the AI responds with a temperature of 22.1462 °C. The human then asks for the average static pressure.

 ------------------ current_messages ------------------
Human Message:  dame la temperatura promedio del computador con tag FQI-EMED-3138.03-057 para el mes de agosto de 2023
AI Message:  La temperatura promedio en agosto de 2023 para la etiqueta de computadora especificada 'FQI-EMED-3138.03-057' fue de 22.1462 °C.
Human Message:  me das su la presion estatica promedio?
AI Message:  La presión estática promedio en el conjunto de datos es de 1527.07 kPa.


In [6]:
assistant.get_current_messages()

[{'type': 'Human',
  'content': 'dame la temperatura promedio del computador con tag FQI-EMED-3138.03-057 para el mes de agosto de 2023'},
 {'type': 'AI',
  'content': "La temperatura promedio en agosto de 2023 para la etiqueta de computadora especificada 'FQI-EMED-3138.03-057' fue de 22.1462 °C."},
 {'type': 'Human', 'content': 'me das su la presion estatica promedio?'},
 {'type': 'AI',
  'content': 'La presión estática promedio en el conjunto de datos es de 1527.07 kPa.'}]